In [37]:
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras_tqdm import TQDMNotebookCallback
from keras.layers import Reshape
from keras import backend as K

In [30]:
# The data, shuffled and split between train and test sets:
classes = 10
img_rows, img_cols = 28, 28
(x_train, train_class), (x_test, test_class) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)


('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


In [55]:
def encoder(datasize=1024):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(datasize))
    model.add(Activation('softmax'))
encoder = encoder()

SyntaxError: non-keyword arg after keyword arg (<ipython-input-55-b111b3bffadb>, line 3)

In [54]:
def decoder(datasize=1024):
    model = Sequential()
    model.add(Dense(input_dim=classes, units=datasize))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(Activation('tanh'))
    model.add(Reshape((128, 7, 7), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model
decoder = decoder()

model = Sequential()
model.add(encoder)
model.add(decoder)

TypeError: The added layer must be an instance of class Layer. Found: None

In [32]:
model.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner = True)])